# Re_entreno

Añadimos acceso a la carpeta de los dataset del Drive del grupo de proyectos

Importante estar ya logeados en la cuenta del grupo, da problemas si estamos en nuestra cuenta personal y queremos montar el drive de la cuenta del grupo.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importamos las bibliotecas necesarias

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
# Para windows con la CPU:
!pip3 install torch torchvision torchaudio 

# Para Windows con CUDA:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

# Para windows con la CPU:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Para Windows con CUDA:
# !pip3 install torch torchvision torchaudio

# Es posible que la mayoría de dependencias ya esten instaladas ^^

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
!git clone https://github.com/ultralytics/yolov5
# Se deberia haber creado una carpeta en nuestro area de trabajo
# Si ya lo teniamos saldra el siguiente error =>fatal: destination path 'yolov5' already exists and is not an empty directory.

fatal: not a git repository (or any of the parent directories): .git


In [6]:
%cd /content/yolov5
!git pull
%cd ../

/content/yolov5
Already up to date.
/content


In [7]:
!pip install -r yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2 #Para abrir webcam
# cv2.imshow() is disabled in Colab, because it causes Jupyter sessionsto crash, consider using cv2_imshow 
from google.colab.patches import cv2_imshow 
import sys # to access the system

# Esta línea de código establece una variable de entorno llamada KMP_DUPLICATE_LIB_OK con el valor "True", lo que significa que permite la carga de bibliotecas duplicadas en el entorno de Python. Esto es útil para algunas bibliotecas de Machine Learning que usan OpenMP para acelerar el cómputo.
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

***
En la siguiente celda se escogen 100 imagenes para hacer el re-entreno.
 
Además esas imagenes se suben a unas carpetas que creamos en el directorio base de colabs.
***

In [ ]:
img = cv2.imread('/content/drive/MyDrive/IA/Datasets/YoloV5/00001.jpg', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

In [34]:
shutil.rmtree('/content/data')

In [37]:
import os
import random
import shutil

# Creamos los directorios necesarios
os.makedirs('/content/data/images', exist_ok=True)
os.makedirs('/content/data/labels', exist_ok=True)

random.seed(15)

src = "/content/drive/MyDrive/IA/Datasets/YoloV5/"
dest_Image = "/content/data/images/"
dest_Label = "/content/data/labels/"

lista_aleatorios_rellenada = []

n = 0
while n < 100:
  aleatorio = random.randint(0,900)
  aleatorio_rellenado = str(aleatorio).zfill(5)
  src_img = src + str(aleatorio_rellenado) + ".jpg"
  if os.path.exists(src_img):
    if aleatorio_rellenado not in lista_aleatorios_rellenada:
      lista_aleatorios_rellenada.append(aleatorio_rellenado)
      n += 1

print("Con cuantas imagenes se va a entrenar:", len(lista_aleatorios_rellenada))
# print("\n", lista_aleatorios_rellenada)

for num in lista_aleatorios_rellenada:
  src_img = src + str(num) + ".jpg"
  dest_img = dest_Image + str(num) + ".jpg"
  shutil.copy2(src_img, dest_img)
  # print(dest_img)

for num in lista_aleatorios_rellenada:
  src_lab = src + str(num) + ".txt"
  dest_lab = dest_Label + str(num) + ".txt"
  shutil.copy2(src_lab, dest_lab)
  # print(dest_lab)

Con cuantas imagenes se va a entrenar: 100


***
**`Entrenar un modelo desde cero`**
***

Ahora tenemos que crear el archivo dataset.yaml, que contiene las categorias de nuestros pesos y las rutas de nuestras imagenes de entrenamiento, test y validación.

In [ ]:
f= open("yolov5/data/cien_images_señales.yaml","w+")

f.write("path: ../data \ntrain: images \nval: images \n\nnc: 4 \n\nnames: ['prohibitory', 'danger', 'mandatory', 'other']")

f.close()

In [ ]:
import shutil
shutil.copy2("/content/yolov5/requirements.txt", "/content/requirements.txt") # Para que checkee el requirements.txt, necesita estar fuera del repositorio

!cd yolov5 && python train.py --img 640 --batch 16 --epochs 500 --data cien_images_señales.yaml --weights yolov5s.pt

shutil.copy2("/content/yolov5/runs/train", "/content/drive/MyDrive/IA/Datasets/Pesos_Yolov5")

train: weights=yolov5s.pt, cfg=, data=cien_images_señales.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-147-gaa7c45c Python-3.9.16 torch-2.0.0+cu118 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0

***
**`Cargamos nuestro modelo`**
***

In [ ]:
modelo_100_señales = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp10/weights/best.pt', force_reload=True)